In [0]:
keys = spark.read.csv('/FileStore/tables/dev_user_databricks_accessKeys.csv', inferSchema=True, header=True)
Access_Id = keys.first()[0]
Secret_Id = keys.first()[1]

access_key = Access_Id
secret_key = Secret_Id
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", access_key)
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret_key)

aws_region = "us-east-1"
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3." + aws_region + ".amazonaws.com")

In [0]:
df = spark.read.csv('s3://project-bucket-17/Output_KMeans', inferSchema=True, header=True)
print(df.count(), len(df.columns))
df.show(5)

3921 12
+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
|CustomerID|Recency|Frequency|         Monetary|  R|  F|  M|RFMGroup|RFMScore|RFM_Loyalty_Level|Cluster|Color|
+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
|     12346|    325|        1|          77183.6|  1|  1|  5|     115|       7|           Silver|      0|  red|
|     12747|      2|      103|4196.009999999999|  5|  4|  5|     545|      14|          Diamond|      2| blue|
|     12748|      1|     4596|         33719.73|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12749|      3|      199|          4090.88|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12820|      3|       59|           942.34|  5|  4|  4|     544|      13|         Platinum|      2| blue|
+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+

In [0]:
df.printSchema()

root
 |-- CustomerID: integer (nullable = true)
 |-- Recency: integer (nullable = true)
 |-- Frequency: integer (nullable = true)
 |-- Monetary: double (nullable = true)
 |-- R: integer (nullable = true)
 |-- F: integer (nullable = true)
 |-- M: integer (nullable = true)
 |-- RFMGroup: integer (nullable = true)
 |-- RFMScore: integer (nullable = true)
 |-- RFM_Loyalty_Level: string (nullable = true)
 |-- Cluster: integer (nullable = true)
 |-- Color: string (nullable = true)



In [0]:
df.createOrReplaceTempView("customers")

In [0]:
# MySQL Queries
spark.sql('select * from customers').show(5)

+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
|CustomerID|Recency|Frequency|         Monetary|  R|  F|  M|RFMGroup|RFMScore|RFM_Loyalty_Level|Cluster|Color|
+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
|     12346|    325|        1|          77183.6|  1|  1|  5|     115|       7|           Silver|      0|  red|
|     12747|      2|      103|4196.009999999999|  5|  4|  5|     545|      14|          Diamond|      2| blue|
|     12748|      1|     4596|         33719.73|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12749|      3|      199|          4090.88|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12820|      3|       59|           942.34|  5|  4|  4|     544|      13|         Platinum|      2| blue|
+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
o

In [0]:
# How many Distinct customers do we have in our database
spark.sql('select count(distinct CustomerID) as Total_Customers from customers').show()

+---------------+
|Total_Customers|
+---------------+
|           3921|
+---------------+



In [0]:
# What is the Monetary value generated by each customer?
spark.sql('select CustomerID, sum(Monetary) as Monetary_value from customers group by CustomerID').show()

+----------+------------------+
|CustomerID|    Monetary_value|
+----------+------------------+
|     12940|            913.54|
|     13285|2709.1200000000003|
|     13623| 747.7800000000001|
|     13832|52.199999999999996|
|     14450|483.24999999999994|
|     14570|            218.06|
|     15447|            155.17|
|     15619|336.40000000000003|
|     15727| 5178.959999999999|
|     15790|            220.85|
|     15957|428.89000000000004|
|     16339|109.95000000000002|
|     16386|             317.2|
|     16503|1431.9299999999998|
|     16574|451.43999999999994|
|     16861|            173.76|
|     17389|          31833.68|
|     17420|            598.83|
|     17679|1992.1100000000001|
|     18024|            389.78|
+----------+------------------+
only showing top 20 rows



In [0]:
# Who are the top 10% of customers in terms of monetary value?

spark.sql('with customer_revenue as (select CustomerID, sum(Monetary) as total_revenue from customers group by CustomerID ) \
    select CustomerID, total_revenue \
        from (select CustomerID, total_revenue, ntile(10) over(order by total_revenue desc) as percentile from customer_revenue ) as customer_percentiles \
        where percentile = 1').show()


+----------+------------------+
|CustomerID|     total_revenue|
+----------+------------------+
|     18102|          259657.3|
|     17450|194550.78999999998|
|     16446|          168472.5|
|     17511|          91062.38|
|     16029|          81024.84|
|     12346|           77183.6|
|     16684|          66653.56|
|     14096|          65164.79|
|     13694|65039.619999999995|
|     15311|60767.899999999994|
|     13089|          58825.83|
|     17949| 58510.48000000001|
|     15769|          56252.72|
|     15061|          54534.14|
|     14298|51527.299999999996|
|     14088|          50491.81|
|     15749|           44534.3|
|     12931|          42055.96|
|     17841|          40991.57|
|     15098|           39916.5|
+----------+------------------+
only showing top 20 rows



In [0]:
# Who are the Top 10% of customers in terms of RFM Score ?

spark.sql('with customer_revenue as (select CustomerID, sum(RFMScore) as RFM_Score from customers group by CustomerID ) \
    select CustomerID, RFM_Score from ( select CustomerID, RFM_Score, NTILE(10) over(order by RFM_Score asc) as percentile from customer_revenue ) as customer_percentiles where percentile = 1').show()


+----------+---------+
|CustomerID|RFM_Score|
+----------+---------+
|     15447|        3|
|     14148|        3|
|     17775|        3|
|     13508|        3|
|     13898|        3|
|     16565|        3|
|     15070|        3|
|     14760|        3|
|     14437|        3|
|     17639|        3|
|     14589|        3|
|     16738|        3|
|     16913|        3|
|     15733|        3|
|     15262|        3|
|     18113|        3|
|     16078|        3|
|     15083|        3|
|     18050|        3|
|     13101|        3|
+----------+---------+
only showing top 20 rows



In [0]:
# Who are the Least 10% of customers in terms of RFM Score ?

spark.sql('with customer_revenue AS (select CustomerID, SUM(RFMScore) as RFM_Score from customers group by CustomerID ) \
    select CustomerID, RFM_Score from ( select CustomerID, RFM_Score, NTILE(10) over(order by RFM_Score desc) as percentile from customer_revenue ) as customer_percentiles where percentile = 1').show()


+----------+---------+
|CustomerID|RFM_Score|
+----------+---------+
|     17389|       15|
|     14713|       15|
|     13468|       15|
|     16549|       15|
|     17346|       15|
|     17677|       15|
|     14085|       15|
|     13994|       15|
|     17001|       15|
|     14180|       15|
|     14554|       15|
|     14056|       15|
|     14367|       15|
|     16265|       15|
|     16839|       15|
|     15531|       15|
|     16684|       15|
|     16326|       15|
|     13081|       15|
|     14961|       15|
+----------+---------+
only showing top 20 rows



In [0]:
# Identify the top 10 customers with an RFM score of 555:
spark.sql('select * from customers where R=5 and F=5 and M=5 order by RFMScore limit 10').show()

+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
|CustomerID|Recency|Frequency|         Monetary|  R|  F|  M|RFMGroup|RFMScore|RFM_Loyalty_Level|Cluster|Color|
+----------+-------+---------+-----------------+---+---+---+--------+--------+-----------------+-------+-----+
|     12748|      1|     4596|         33719.73|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12749|      3|      199|          4090.88|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12839|      2|      314|          5591.42|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12841|      4|      420|          4022.35|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12856|      7|      314|          2179.93|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|     12921|      9|      720|         16587.09|  5|  5|  5|     555|      15|          Diamond|      2| blue|
|

In [0]:
# Count the number of customers in each RFM Segment :
spark.sql('select concat(R, F, M) as rfm_segment, count(*) as customer_count from customers group by rfm_segment order by rfm_segment desc').show()

+-----------+--------------+
|rfm_segment|customer_count|
+-----------+--------------+
|        555|           283|
|        554|            67|
|        553|            18|
|        552|             2|
|        545|            65|
|        544|            78|
|        543|            25|
|        542|            14|
|        541|             3|
|        535|            15|
|        534|            32|
|        533|            44|
|        532|            19|
|        531|            10|
|        525|             3|
|        524|            12|
|        523|            16|
|        522|            29|
|        521|             7|
|        515|             3|
+-----------+--------------+
only showing top 20 rows

